<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/phi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def init_model():
    model_name = "microsoft/phi-2" # Replace with the actual model name on Hugging Face Hub
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [4]:
import os
import numpy as np

def save_layer_activations(hidden_states, layer_index, output_dir="layer_activations", filename="activations.txt"):
    """
    Saves the hidden states of a specific layer to a text file.

    Args:
        hidden_states (tuple): Tuple of hidden states from all layers.
        layer_index (int): The index of the layer to save activations from.
        output_dir (str): The directory to save the output file.
        filename (str): The name of the output file.
    """
    if layer_index < len(hidden_states):
        layer_activations = hidden_states[layer_index]

        # Assuming batch size is 1 and sequence length is the second dimension
        # The shape is typically (batch_size, sequence_length, hidden_size)
        # We need to save activations for each token in the sequence

        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        file_path = os.path.join(output_dir, filename)

        # Open in append mode to write activations from different inference calls
        with open(file_path, 'a') as f:
            # Iterate through each token in the sequence
            # Assuming batch size is 1, we access layer_activations[0]
            for token_activations in layer_activations[0].cpu().detach().numpy():
                np.savetxt(f, token_activations.reshape(1, -1), fmt='%f') # Use numpy.savetxt, reshape for 1D array
                f.write('\n') # Add a newline after each token's activations

        print(f"Activations from layer {layer_index} saved to {file_path}")
    else:
        print(f"Error: Layer index {layer_index} is out of bounds.")




def run_inference(model, tokenizer, input_prompt, activations_filename):
    input_text = input_prompt
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    print(f"tokenizer.eos_token_id : {tokenizer.eos_token_id}")
    # Configure the model to return hidden states
    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model(input_ids, output_hidden_states=True)
        generated_ids = model.generate(input_ids, max_length=80) # You can adjust max_length




    # To get the activations of a specific layer (e.g., the 5th layer):
    #layer_index = 5 # Choose the index of the layer you want
    #activations = hidden_states[layer_index]
    #print(f"Shape of activations from layer {layer_index}: {activations.shape}")
    # Decode and print the generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print("\nGenerated text:")
    print(generated_text)



    whole_text = input_text + generated_text
    whole_text_ids = tokenizer.encode(whole_text, return_tensors="pt")

    with torch.no_grad(): # Disable gradient calculation for inference
        wholetext_outputs = model(whole_text_ids, output_hidden_states=True)

    # Access the hidden states
    # outputs.hidden_states is a tuple where each element is the hidden state for a layer
    # The first element is the embedding layer output, and the last is the output before the classification head
    hidden_states = wholetext_outputs.hidden_states

    print("Shapes of hidden states for all layers:")
    for i, layer_hidden_state in enumerate(hidden_states):
        print(f"  Layer {i}: {layer_hidden_state.shape}")
        #print(f"  Layer {i}: {layer_hidden_state}")

    save_layer_activations(hidden_states, 32, "layer_activations", activations_filename)





In [3]:
model, tokenizer = init_model()

#run_inference(model, tokenizer, "Once upon a time")
#run_inference(model, tokenizer, "please write fibonacci function in php")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [2]:
#run_inference(model, tokenizer, "Your input text here")

run_inference(model, tokenizer, "please write fibonacci function in c", "activations_phi2_factor_c.txt")

NameError: name 'model' is not defined